# Lab 4: AgentCore Runtime으로 프로덕션 배포

## 개요

이전 랩들에서 메모리, 공유 도구, 프로덕션급 배포를 갖춘 포괄적인 이커머스 고객 지원 에이전트를 구축했습니다. AgentCore 서비스의 기능을 보여주어 에이전트 사용 사례를 프로토타입에서 프로덕션으로 마이그레이션하는 과정을 시연했습니다. 이제 실제 고객 대화를 처리할 수 있는 시스템이 준비되었습니다.

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html)은 프레임워크, 프로토콜 또는 모델 선택에 관계없이 조직이 프로덕션에서 AI 에이전트를 배포하고 확장할 수 있도록 지원하는 안전하고 완전히 관리되는 런타임입니다. 엔터프라이즈급 안정성, 자동 확장 및 포괄적인 모니터링 기능을 제공합니다.

**워크숍 여정:**

- **Lab 1 (완료)**: 에이전트 프로토타입 생성 - 기능적인 이커머스 고객 지원 에이전트 구축
- **Lab 2 (완료)**: 메모리로 강화 - 대화 컨텍스트 및 개인화 추가
- **Lab 3 (완료)**: Gateway & Identity로 확장 - 에이전트 간 도구 안전하게 공유
- **Lab 4 (현재)**: 프로덕션 배포 - AgentCore Runtime 배포
- **Lab 5**: AgentCore 관측성 - CloudWatch GenAI 관측성으로 모니터링
- **Lab 6**: 사용자 인터페이스 구축 - 고객 대상 애플리케이션 생성

### AgentCore Runtime & 프로덕션 배포의 중요성

현재 상태 (Lab 1-3): 에이전트가 로컬에서 중앙 집중식 도구로 실행되지만 프로덕션 과제에 직면:

- 에이전트가 단일 세션에서 로컬로 실행됨
- 포괄적인 모니터링 또는 디버깅 기능 없음
- 여러 동시 사용자를 안정적으로 처리할 수 없음

이 랩 후에는 다음과 같은 프로덕션 준비 에이전트 인프라를 갖게 됩니다:

- 가변 수요를 처리하는 서버리스 자동 확장
- 추적, 메트릭 및 로깅을 통한 포괄적인 관측성
- 자동 오류 복구를 통한 엔터프라이즈 안정성
- 적절한 액세스 제어를 통한 보안 배포
- AWS 콘솔 및 API를 통한 쉬운 관리 및 실제 프로덕션 워크로드 지원

### Lab 4를 위한 아키텍처
<div style="text-align:left"> 
    <img src="../images/architecture_lab4_runtime.png" width="75%"/> 
</div>

*CloudWatch를 통한 완전한 관측성을 갖춘 AgentCore Runtime에서 실행되는 에이전트로, 자동 확장 및 포괄적인 모니터링을 통해 프로덕션 트래픽을 처리합니다. 이전 랩의 메모리 및 게이트웨이 통합은 프로덕션 환경에서 완전히 기능합니다.*

### 주요 기능

- **서버리스 에이전트 배포:** 최소한의 코드 변경으로 로컬 에이전트를 확장 가능한 프로덕션 서비스로 변환
- **Cognito 인증:** JWT 토큰 기반 사용자 인증으로 보안 강화


**참고**: AgentCore 관측성 추적을 CloudWatch에서 보려면 [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html)를 활성화해야 합니다.

### Step 1: 필수 라이브러리 가져오기

In [ ]:
# 필수 라이브러리 가져오기
import os
import json
import boto3
from strands import Agent
from strands.models import BedrockModel

# 경로 설정 (lab-04-agentcore-runtime/ 폴더에서 프로젝트 루트로)
import sys; sys.path.insert(0, '../..'); from src.helpers.ecommerce_memory import create_or_get_ecommerce_memory_resource

create_or_get_ecommerce_memory_resource()  # 메모리 랩이 실행되지 않은 경우를 대비

### Step 2: AgentCore Runtime용 에이전트 준비

#### Runtime 호환 에이전트 생성

Python SDK를 통해 이전 로컬 에이전트 구현 내에서 필요한 AgentCore Runtime 구성 요소를 먼저 정의해보겠습니다.

아래 `#### AGENTCORE RUNTIME - LINE i ####` 주석을 보면 런타임 준비 에이전트를 준비하는 관련 배포 코드가 추가된 위치를 확인할 수 있습니다. 런타임 준비 에이전트를 준비하는 4개의 라인을 찾을 수 있습니다:

1. `from bedrock_agentcore.runtime import BedrockAgentCoreApp`으로 Runtime App 가져오기
2. `app = BedrockAgentCoreApp()`으로 App 초기화
3. `@app.entrypoint`로 호출 함수 데코레이트
4. `app.run()`으로 AgentCore Runtime이 실행을 제어하도록 함

In [ ]:
%%writefile lab4_runtime.py
"""
AgentCore Runtime용 이커머스 고객 지원 에이전트
도구를 ecommerce_tools.py에서 import하여 사용합니다.
"""
from bedrock_agentcore.runtime import (
    BedrockAgentCoreApp,
)  #### AGENTCORE RUNTIME - LINE 1 ####
from strands import Agent
from strands.models import BedrockModel

# 같은 디렉토리의 ecommerce_tools에서 도구와 설정 import
from ecommerce_tools import (
    check_return_eligibility,
    process_return_request,
    get_product_recommendations,
    ECOMMERCE_SYSTEM_PROMPT,
    ECOMMERCE_MODEL_ID,
)

# ============================================================
# 에이전트 및 런타임 앱 설정
# ============================================================
model = BedrockModel(model_id=ECOMMERCE_MODEL_ID)

agent = Agent(
    model=model,
    tools=[check_return_eligibility, process_return_request, get_product_recommendations],
    system_prompt=ECOMMERCE_SYSTEM_PROMPT,
)

app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####


@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
def invoke(payload):
    """AgentCore Runtime 엔트리포인트 함수"""
    user_input = payload.get("prompt", "")
    response = agent(user_input)
    return response.message["content"][0]["text"]


if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####

### Step 3: AgentCore Runtime에 배포

이제 [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit)을 사용하여 에이전트를 AgentCore Runtime에 배포하겠습니다.

#### 보안 Runtime 배포 구성 (AgentCore Runtime + AgentCore Identity)

먼저 스타터 툴킷을 사용하여 엔트리포인트, 실행 역할, requirements 파일과 함께 AgentCore Runtime 배포를 구성합니다. 또한 Amazon Cognito 사용자 풀을 사용하여 신원 인증을 구성하고 시작 시 Amazon ECR 리포지토리를 자동 생성하도록 스타터 킷을 구성합니다.

#### Lab 3 vs Lab 4 Cognito 차이점

| 구분 | Lab 3 (Gateway) | Lab 4 (Runtime) |
|------|-----------------|-----------------|
| **목적** | Agent → Gateway 인증 | User → Runtime 인증 |
| **Auth Flow** | Client Credentials Grant | User Password Auth |
| **용도** | M2M (서버 간 통신) | 사용자 인증 |
| **생성 방식** | CloudFormation | 코드에서 생성/재사용 |

#### 인증 흐름 비교

**Lab 3: Client Credentials Grant**
```
Agent              Cognito                 Gateway
  │                   │                       │
  │── client_id ─────>│                       │
  │── client_secret ─>│                       │
  │                   │                       │
  │<── access_token ──│                       │
  │    (JWT)          │                       │
  │                   │                       │
  │───────────────── JWT ────────────────────>│
```

**Lab 4: User Password Auth**
```
Client             Cognito                 Runtime
  │                   │                       │
  │── client_id ─────>│                       │
  │── client_secret ─>│                       │
  │── username ──────>│                       │
  │── password ──────>│                       │
  │                   │                       │
  │<── access_token ──│                       │
  │    (JWT)          │                       │
  │                   │                       │
  │───────────────── JWT ────────────────────>│
```

**JWT 내용 차이:**
| | Lab 3 JWT | Lab 4 JWT |
|---|---|---|
| **포함 정보** | client_id만 | client_id + **username** |
| **식별** | "어떤 앱?" | "어떤 앱?" + "**어떤 사용자?**" |

Lab 4의 JWT는 사용자 ID(`username`)를 포함하여 Runtime이 사용자별 세션을 추적할 수 있습니다.

> **💡 참고**: Lab 3에서 CloudFormation으로 생성한 Cognito는 M2M(Machine-to-Machine) 인증용입니다. Lab 4에서는 사용자 인증이 필요하므로 별도의 User Pool을 사용합니다. 기존 설정이 있으면 자동으로 재사용합니다.

**참고**: Cognito access_token은 2시간만 유효합니다. access_token이 만료되면 `reauthenticate_user` 메서드를 사용하여 새 access_token을 발급받을 수 있습니다.

In [ ]:
# Runtime용 Cognito 사용자 인증 설정
# 참고: Lab 3의 Gateway Cognito (M2M)와 다른 용도입니다
# - Lab 3: Agent → Gateway (Client Credentials Grant, 서버 간 통신)
# - Lab 4: User → Runtime (User Password Auth, 사용자 인증)

from src.helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("🔐 Runtime용 Cognito 사용자 인증 설정 중...")
print("=" * 60)

# 기존 설정이 있으면 재사용, 없으면 새로 생성
cognito_config = setup_cognito_user_pool()
print("\ncognito_config:\n", json.dumps(cognito_config, indent=2))

print("=" * 60)
print(f"✅ Cognito 설정 완료!")
print(f"   Discovery URL: {cognito_config['discovery_url']}")
print(f"   Bearer Token 길이: {len(cognito_config['bearer_token'])} chars")
print()
print("💡 참고: 토큰은 2시간 후 만료됩니다.")
print("   만료 시 reauthenticate_user() 함수로 재발급 가능합니다.")

### Step 4: Runtime 배포 구성

AgentCore Starter Toolkit을 사용하여 Runtime 배포를 구성합니다. 실행 역할, ECR 리포지토리, Cognito 인증 등을 설정합니다.

#### 노트북 디렉토리 구조

모든 Runtime 관련 파일이 `notebooks/` 디렉토리에 위치합니다:

```
notebooks/
├── lab4_runtime.py      ← 엔트리포인트
├── ecommerce_tools.py   ← 도구 모듈
└── requirements.txt     ← 의존성 파일
```

#### 자동 생성되는 파일

`configure()` 명령은 다음 파일들을 현재 디렉토리에 자동 생성합니다:

| 파일 | 설명 |
|------|------|
| `.bedrock_agentcore.yaml` | 에이전트 배포 설정 (엔트리포인트, IAM 역할, 인증 등) |
| `Dockerfile` | 에이전트를 컨테이너화하기 위한 Docker 빌드 파일 |
| `.dockerignore` | Docker 빌드 시 제외할 파일 목록 |

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from src.helpers.utils import create_agentcore_runtime_execution_role
import os

# 현재 디렉토리 확인 (lab-04-agentcore-runtime 폴더에서 실행)
print(f"📂 작업 디렉토리: {os.getcwd()}")

# 기존 설정 파일 삭제
if os.path.exists(".bedrock_agentcore.yaml"):
    os.remove(".bedrock_agentcore.yaml")
    print("기존 설정 파일 삭제 완료")

# Runtime 툴킷 초기화
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = create_agentcore_runtime_execution_role()

# 완전히 새로운 Runtime 인스턴스 생성
agentcore_runtime = Runtime()

# 새로운 에이전트 이름으로 배포 구성 (Cognito 인증 포함)
import uuid
agent_suffix = str(uuid.uuid4())[:8]
agent_name = f"ecommerce_test_{agent_suffix}"

# Cognito 인증과 함께 구성
# 모든 파일이 lab-04-agentcore-runtime/ 디렉토리에 위치
response = agentcore_runtime.configure(
    entrypoint="lab4_runtime.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
)

print("✅ Runtime 구성 완료 (Cognito 인증 포함)")
print(f"📋 새 에이전트 이름: {agent_name}")
print(f"🔐 Cognito Client ID: {cognito_config.get('client_id')}")
print(f"🌐 Discovery URL: {cognito_config.get('discovery_url')}")

### Step 5: 에이전트 배포

구성이 완료되었으므로 이제 에이전트를 AgentCore Runtime에 배포합니다. `launch()` 명령은 다음 작업을 수행합니다:

1. Docker 이미지 빌드
2. ECR에 이미지 푸시
3. AgentCore Runtime에 에이전트 등록
4. 엔드포인트 생성 및 시작

> **⏱️ 참고**: 첫 배포는 Docker 이미지 빌드로 인해 시간이 소요될 수 있습니다.

In [ ]:
# Import 에러 수정 후 새 에이전트 배포
from src.helpers.utils import put_ssm_parameter

print("Import 에러 수정 후 새 에이전트 배포 중...")
launch_result = agentcore_runtime.launch()
print("배포 완료:", launch_result.agent_arn)

agent_arn = put_ssm_parameter(
    "/app/ecommerce/agentcore/runtime_arn", launch_result.agent_arn
)

AgentCore Runtime 생성:
![agentcore_runtime.png](../images/agentcore_runtime.png)

### Step 6: 프로덕션 준비 상태 확인

에이전트가 프로덕션 트래픽을 처리할 준비가 되었는지 확인하겠습니다.

In [ ]:
# 배포 완료까지 대기
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"배포 대기 중... 현재 상태: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"최종 상태: {status}")

### Step 7: 에이전트 테스트 - 세션 연속성 확인

배포된 에이전트를 테스트하여 세션 연속성이 올바르게 작동하는지 확인합니다.

**테스트 시나리오:**
1. **첫 번째 요청**: 반품 문의 (세션 시작)
2. **두 번째 요청**: 같은 세션 ID로 추가 정보 제공 → 이전 대화 맥락 유지 확인
3. **세 번째 요청**: 새로운 세션 ID로 요청 → 이전 맥락 없이 새 대화 시작 확인

`invoke_agent_with_response()` 유틸리티 함수는 스트리밍/일반 모드를 자동 전환하여 응답을 처리합니다.

In [ ]:
# 세션 연속성 시연을 위한 세션 ID 생성
import uuid
import importlib
import src.helpers.utils as utils_module
importlib.reload(utils_module)  # 모듈 다시 로드 (새로운 함수 적용)
from src.helpers.utils import invoke_agent_with_response

session_id = uuid.uuid4()

# 이커머스 고객 지원 시나리오 테스트
user_query = "지난주에 주문한 원피스 사이즈가 안 맞아서 반품하고 싶어요."
print("user_query:", user_query)

bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

print("🤖 K-Style 고객센터 에이전트가 답변 중입니다...")
print("-" * 60)

response = invoke_agent_with_response(
    runtime=agentcore_runtime,
    prompt=user_query,
    bearer_token=bearer_token,
    session_id=str(session_id),
    title="에이전트 응답"
)

print("-" * 60)
print("✅ 응답 완료")

In [ ]:
# 세션 연속성 테스트 - 같은 세션 ID로 추가 정보 제공
user_query = "주문번호는 ORD-20240810-001이고, 상품명은 '플라워 패턴 원피스'입니다."
print("user_query:", user_query)

print("🔄 같은 세션에서 추가 정보 제공 중...")
print("-" * 60)

response = invoke_agent_with_response(
    runtime=agentcore_runtime,
    prompt=user_query,
    bearer_token=bearer_token,
    session_id=str(session_id),  # 같은 세션 ID 사용
    title="세션 연속성 응답"
)

print("-" * 60)
print("✅ 세션 연속성 확인 완료 - 이전 대화 맥락을 기억하고 있습니다!")

In [ ]:
# 새로운 고객을 시연하기 위한 새 세션 ID 생성
session_id2 = uuid.uuid4()

user_query = "여전히 작동하지 않아요. 무슨 일이 일어나고 있는 거죠?"
print("user_query:", user_query)

print("👤 새로운 고객 세션 시작...")
print("🤖 에이전트가 새로운 맥락에서 응답 중...")
print("-" * 60)

response = invoke_agent_with_response(
    runtime=agentcore_runtime,
    prompt=user_query,
    bearer_token=bearer_token,
    session_id=str(session_id2),  # 새로운 세션 ID 사용
    title="새 고객 응답"
)

print("-" * 60)
print("✅ 새 세션 확인 완료 - 이전 대화 맥락을 기억하지 않고 새로운 대화로 시작!")

이 경우 에이전트는 더 이상 맥락을 가지고 있지 않으며 더 많은 정보가 필요합니다.

이것이 기반 인프라를 관리할 필요 없이 에이전트를 위한 안전하고 확장 가능한 엔드포인트를 갖는 데 필요한 모든 것입니다!

### 🎉 축하합니다!

**Lab 4: 프로덕션 배포 - AgentCore Runtime 사용**을 성공적으로 완료했습니다!

달성한 성과:

##### 프로덕션급 배포:

- 최소한의 코드 변경으로 에이전트를 프로덕션에 준비 (단 4줄만 추가)
- 서로 다른 고객 간 적절한 세션 격리 검증
- 세션별 연속성 + 메모리 지속성 및 컨텍스트 인식 확인

##### 엔터프라이즈급 보안 & 신원 관리:

- Cognito 통합을 통한 JWT 토큰 기반 보안 인증 구현
- 프로덕션 워크로드를 위한 적절한 IAM 역할 및 실행 권한 구성
- 보안 에이전트 호출을 위한 신원 기반 액세스 제어 설정

##### 다음 단계 [Lab 5: AgentCore 관측성 →](../lab-05-agentcore-observability/lab-05-agentcore-observability.ipynb)
Lab 5에서는 CloudWatch GenAI 관측성을 사용하여 에이전트의 성능을 모니터링하고 디버깅하는 방법을 배웁니다!